In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth
# c_path = "/content/outputs/checkpoint-282"
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.12.11: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=["q_proj", "k_proj", "v_proj", "up_proj", "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth"
)

Unsloth 2024.12.11 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
from transformers import TrainingArguments
from datasets import load_dataset
from trl import SFTTrainer

In [17]:
data_files = {"train": "/content/train.csv", "validation": "/content/val.csv", "test": "/content/test.csv"}
dataset = load_dataset("csv", data_files=data_files)

In [18]:
# classification_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# Carefully read the following post and classify it as 'Yes' (contains signs of belongingness) or 'No' (does not contain signs of belongingness) only.

# ### Input:
# {}

# ### Response:
# {}
# """

In [ ]:
classification_prompt = """
**Context**:
- You are a highly experienced psychology expert specializing in belongingness assessment.

**Instructions**:
- Carefully read the given post (somewhere on the Internet).
- Base your decision on the explicit content of the post. Avoid assumptions beyond what is stated.
- Your task is to classify the given post into 'Yes' or 'No' belongingness.

**Post for Classification**:
"{}"

**Output Format**:
- Do not provide any additional explanation or commentary or punctuation. Only output is Yes or No.

**Respone**:
"{}"
"""

In [7]:
EOS_TOKEN = tokenizer.eos_token

In [19]:
def format_data(examples):
    posts = examples["post"]
    labels = examples["is_belong"]
    texts = [
        classification_prompt.format(post, label) + EOS_TOKEN
        for post, label in zip(posts, labels)
    ]
    return {"text": texts}

In [20]:
dataset = dataset.map(format_data, batched=True)

Map:   0%|          | 0/1972 [00:00<?, ? examples/s]

Map:   0%|          | 0/493 [00:00<?, ? examples/s]

Map:   0%|          | 0/1057 [00:00<?, ? examples/s]

In [21]:
def convert_labels(examples):
    examples["labels"] = [1 if label == "Yes" else 0 for label in examples["is_belong"]]
    return examples

In [22]:
dataset = dataset.map(convert_labels, batched=True)

Map:   0%|          | 0/1972 [00:00<?, ? examples/s]

Map:   0%|          | 0/493 [00:00<?, ? examples/s]

Map:   0%|          | 0/1057 [00:00<?, ? examples/s]

In [14]:
print(dataset["train"][0])

{'post': "All my life i've been going through shit (only 17 years old) and when things started to get better i crashed. I can't get myself to get out of bed no matter how much i try, my family understands but do still not approve since my grades dropped from all A's to E-C. It has been like this for 1-2 years now and none of my friends understands how It's like, I can't really blame them either since I don't like talking about it and i've always been taught to be a man and keep this stuff to myself. They just see a lazy fuck who is too irresponsible to go too school, same with my teachers. Idk if typing here is going to help at all but if anyone has some tips/advice on how to get motivated again i would be super happy.", 'is_belong': 'Yes', 'text': "Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nCarefully read the following post and classify it as 'Yes' (c

In [23]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    args=TrainingArguments(
        output_dir="outputs",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        num_train_epochs=3,
        fp16=True,
        logging_steps=10,
        optim = "adamw_8bit",
        save_strategy="epoch",
        save_total_limit=2,
        weight_decay=0.01,
        seed=42,
    ),
)

Map:   0%|          | 0/1972 [00:00<?, ? examples/s]

Map:   0%|          | 0/493 [00:00<?, ? examples/s]

In [24]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 1,972 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 369
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.475000
20,1.573100
30,1.632000
40,1.665000
50,1.640700
60,1.674200
70,1.631400
80,1.620000
90,1.621200
100,1.640200


TrainOutput(global_step=369, training_loss=1.383637242200898, metrics={'train_runtime': 5092.4239, 'train_samples_per_second': 1.162, 'train_steps_per_second': 0.072, 'total_flos': 8.586856270184448e+16, 'train_loss': 1.383637242200898, 'epoch': 2.9817444219066935})

### Evaluate

In [25]:
FastLanguageModel.for_inference(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [ ]:
inputs = tokenizer(
[
    classification_prompt.format(
        "[I'm not seeking any medical advice, just want to know if anyone has had similar experiences] I've just started taking Lexapro for moderate depression and anxiety. For the first week I was prescribed 5mg, 10mg thereafter. It's been two days and I have this mild feeling like I'm on ecstasy/MDMA. My body feels light and I have this weird concentration of energy above my neck at the back of my head. Has anyone felt this/experienced this?",
        "",
    )
], return_tensors = "pt").to("cuda")

In [ ]:
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [26]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm

In [27]:
batch_size = 4
valid_texts = dataset["test"]["post"]
valid_labels = dataset["test"]["is_belong"]
valid_predictions = []

In [28]:
for i in tqdm(range(0, len(valid_texts), batch_size), desc="Processing Test Set"):
    batch_posts = valid_texts[i : i + batch_size]

    batch_inputs = [
        classification_prompt.format(post, "") for post in batch_posts
    ]

    inputs = tokenizer(
        batch_inputs,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    decoded_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    clean_predictions = [
        output.strip().split("\n")[-1]
        for output in decoded_outputs
    ]
    valid_predictions.extend(clean_predictions)

Processing Test Set: 100%|██████████| 265/265 [06:47<00:00,  1.54s/it]


In [29]:
print("Sample Predictions:")
for i, pred in enumerate(valid_predictions[:5]):
    print(f"Post {i + 1}: {valid_texts[i]}")
    print(f"Prediction: {pred}")
    print(f"True Label: {valid_labels[i]}")
    print("---")

Sample Predictions:
Post 1: [I'm not seeking any medical advice, just want to know if anyone has had similar experiences] I've just started taking Lexapro for moderate depression and anxiety. For the first week I was prescribed 5mg, 10mg thereafter. It's been two days and I have this mild feeling like I'm on ecstasy/MDMA. My body feels light and I have this weird concentration of energy above my neck at the back of my head. Has anyone felt this/experienced this?
Prediction: No
True Label: No
---
Post 2: Im afraid. The suicide scale is true. Its gets progressively worse and you dont even realize it. I was at 1 contemplating death and that was it. Now as i live i keep moving farther into the scale. Not existing is all i long for. Its all i think about. Its all i desire. Ive written 2 goodbye notes. I constantly open up the lid of pills and acting as if ill swallow them. Yesterday and today i have been making nooses with a thin belt and putting it around my neck. I squeeze so hard to simu

In [30]:
binary_predictions = [1 if pred == "Yes" else 0 for pred in valid_predictions]
binary_labels = [1 if label == "Yes" else 0 for label in valid_labels]

accuracy = accuracy_score(binary_labels, binary_predictions)
report = classification_report(binary_labels, binary_predictions, target_names=["No", "Yes"])

print(report)

              precision    recall  f1-score   support

          No       0.61      0.57      0.59       472
         Yes       0.67      0.70      0.69       585

    accuracy                           0.65      1057
   macro avg       0.64      0.64      0.64      1057
weighted avg       0.64      0.65      0.64      1057

